In [266]:
import random

def roll_dice(dice_str, explode, advantage=0):
    try:
        # Split the input string into number of dice and number of faces
        num_dice, num_faces = map(int, dice_str.split('d'))

        # Roll the dice and return the results
        results = []

        for roll_num in range(num_dice):
            # Roll a die with the given number of faces

            if roll_num == 0:
                num_dice = abs(advantage) + 1
                result_list = [random.randint(1, num_faces) for _ in range(num_dice)]

                if advantage > 0:
                    result = max(result_list)
                elif advantage < 0:
                    result = min(result_list)
                else:
                    result = result_list[0]

                # If explode is True and the result equals the number of faces, keep rolling
                while explode and result == num_faces:
                    results.append(result)  # Record the result
                    result = random.randint(1, num_faces)
            else:
                result = random.randint(1, num_faces)

            results.append(result)  # Record the final result

        return results
    
    except ValueError:
        # Handle the case where the input values are not valid
        return "Invalid input values. Please use integers for num_dice and num_faces."
    
def attack_roll(dice_str, damage_threshold, damage_modifier):
    # Roll the dice using the roll_dice function
    dice_results = roll_dice(dice_str, True)
    print(dice_results)

    # Check if the first value is less than the damage threshold
    if dice_results[0] < damage_threshold:
        return 0  # No damage done

    # Initialize total damage with the sum of dice results and the damage modifier
    total_damage = sum(dice_results) + damage_modifier

    return total_damage

# Example usage:
print(roll_dice("2d4", True, 3))

damage = attack_roll("1d4", damage_threshold=1, damage_modifier=5)
print(damage)

[4, 1, 1]
[4, 2]
11


In [267]:
class Player:
    def __init__(self, hp, ac, num_attacks, attack_dice, attack_modifier):
        self.hp = hp
        self.ac = ac
        self.num_attacks = num_attacks
        self.attack_dice = attack_dice
        self.attack_modifier = attack_modifier

    def attack_turn(self, target, print_results=False):
        total_damage = 0

        # Process normal attacks
        for _ in range(self.num_attacks):
            die_rolls = roll_dice(self.attack_dice, True)

            if print_results:
                print("Player attack roll: " + str(die_rolls))

            damage = target.process_attack_roll(die_rolls, self.attack_modifier, print_results)

            if print_results:
                print("Player did " + str(damage) + " damage")
                
            total_damage += damage

        # Process one more attack with disadvantage
        die_rolls = roll_dice(self.attack_dice, True, -1)

        if print_results:
            print("Player attack roll (disadvantage): " + str(die_rolls))

        damage = target.process_attack_roll(die_rolls, self.attack_modifier, print_results)

        if print_results:
            print("Player did " + str(damage) + " damage")
            
        total_damage += damage

        if print_results:
            print("Player did " + str(total_damage) + " damage this round")
        
        return total_damage

    def get_block_amount(self):
        return self.ac - 8

In [268]:
class Monster:
    def __init__(self, hp, damage_threshold, num_attacks, attack_dice, attack_modifier):
        self.hp = hp
        self.damage_threshold = damage_threshold
        self.num_attacks = num_attacks
        self.attack_dice = attack_dice
        self.attack_modifier = attack_modifier

    def attack_turn(self, target, print_results=False):
        used_block = False
        total_damage = 0
        for _ in range(self.num_attacks):
            die_rolls = roll_dice(self.attack_dice, True)
            if print_results:
                print("Monster attack roll: " + str(die_rolls))
            if die_rolls[0] > 1:
                damage = sum(die_rolls) + self.attack_modifier
              
                if not used_block:
                    damage_after = max(0, damage - target.get_block_amount())
                    if print_results:
                        print("Monster did " + str(damage) + " damage but Player blocked " + str(target.get_block_amount()))
                    damage = damage_after
                    used_block = True
                else:
                    if print_results:
                        print("Monster did " + str(damage) + " damage")
                              
                total_damage += damage
            else:
                if print_results:
                    print("Monster missed")

        if print_results:
            print("Monster did " + str(total_damage) + " damage this round")
            
        return total_damage

    def process_attack_roll(self, die_results, damage_modifier, print_results=False):
        # Check if the first member of die_results is less than damage_threshold
        if die_results[0] <= self.damage_threshold:
            if print_results:
                print("Roll was below damage threshold of " + str(self.damage_threshold))
            return 0  # No damage done

        # Calculate the total damage by summing die_results and adding damage_modifier
        total_damage = sum(die_results) + damage_modifier
        return total_damage

In [269]:
fighter = Player(hp=40, ac=18, num_attacks=1, attack_dice="1d8", attack_modifier=3)
monster = Monster(hp=34, damage_threshold=1, num_attacks=2, attack_dice="1d10", attack_modifier=4)

In [270]:
print(fighter.attack_turn(monster, print_results=True))

Player attack roll: [6]
Player did 9 damage
Player attack roll (disadvantage): [2]
Player did 5 damage
Player did 14 damage this round
14


In [274]:
print(monster.attack_turn(fighter, print_results=True))

Monster attack roll: [9]
Monster did 13 damage but Player blocked 10
Monster attack roll: [2]
Monster did 6 damage
Monster did 9 damage this round
9


In [275]:
def get_dpr(player, monster, num_simulations=100000):
    average_damage = 0.0
    for _ in range(num_simulations):
        damage = player.attack_turn(monster)
        average_damage += damage / num_simulations
    return average_damage

In [288]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Player(hp=40, ac=18, num_attacks=1, attack_dice="1d8", attack_modifier=3)
monster = Monster(hp=34, damage_threshold=1, num_attacks=2, attack_dice="1d10", attack_modifier=4)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr = get_dpr(player, monster)
player_turns = monster.hp / player_dpr
monster_dpr = get_dpr(monster, player)
monster_turns = player.hp / monster_dpr

print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))


Player DPR: 12.984789999998542
Monster DPR: 10.43724000000032
Player Turns to Win: 2.61844819977865
Monster Turns to Win: 3.8324307958807857
